## Gans Challenge - CITIES & WEATHER database 😀


A complete recap of this project is found in the following **[Medium article](https://medium.com/@marina.l./data-engineering-is-for-engineers-not-49fe0dc22497)**.


---
## 1.&nbsp; Make a loop to retrieve all the other details we want from Wikipedia & create a dataframe

In [3]:
# make sure we've installed mysql to connect
#!pip install sqlalchemy
#!pip install pymysql

In [1]:
# import necessary libraries
import pandas as pd
import requests   
from bs4 import BeautifulSoup 
from datetime import datetime       # we need this to fill in the "year retrieved" column
from pytz import timezone

In [2]:
cities_list = ["Berlin", "Hamburg", "Munich", "Cologne", "Frankfurt"]
states = []
latitudes = []
longitudes = []

for city in cities_list:
  url = f"https://www.wikipedia.org/wiki/{city}"   # it turns the url into an f string and puts the city as a variable so it changes accordingly from one city to the other
  response = requests.get(url)          # gets all the content from the wikipedia page and saves it under response
  city_soup = BeautifulSoup(response.content, 'html.parser')    # parses the content (saves the content of the wikipedia site under the city_soup variable)

  # retrieve the state the city belongs to
  if city not in ["Hamburg", "Berlin"]:         # for Berlin, the general .find formula would work too! it just Hamburg that does not have a "State" section
    city_state = city_soup.find("table", class_="vcard").find(string="State").find_next("td").get_text()  # retrieves the state for cities with different state name
  else:
    city_state = city     # for Hamburg and Berlin, it just takes the homonymous city name
  states.append(city_state)

  # retrieve latitude of each city and add it to the column latitudes
  city_latitude = city_soup.find(class_="latitude").get_text()
  latitudes.append(city_latitude)  

   # retrieve longitude of each city and add it to the column latitudes
  city_longitude = city_soup.find(class_="longitude").get_text()
  longitudes.append(city_longitude)  


cities_info_non_rel = pd.DataFrame({         # this will be the cities_info dataframe
    "city_name": cities_list,
    "german_state": states,
    "latitude": latitudes,
    "longitude": longitudes
})  

display(cities_info_non_rel)    # instead of print, it shows the table in a prettier table format




,city_name,german_state,latitude,longitude
0,Berlin,Berlin,52°31′12″N,13°24′18″E
1,Hamburg,Hamburg,53°33′N,10°00′E
2,Munich,Bavaria,48°08′15″N,11°34′30″E
3,Cologne,North Rhine-Westphalia,50°56′11″N,6°57′10″E
4,Frankfurt,Hesse,50°06′38″N,08°40′56″E


In [3]:
cities_info_non_rel.dtypes              # LATER ON: I may have to change the format of latitude/longitude without special characters

city_name       object
german_state    object
latitude        object
longitude       object
dtype: object

In [4]:
populations = []
year_data_retrieved = []

for city in cities_list:
  url = f"https://www.wikipedia.org/wiki/{city}"   # it turns the url into an f string and puts the city as a variable so it changes accordingly from one city to the other
  response = requests.get(url)          # gets all the content from the wikipedia page and saves it under response
  city_soup = BeautifulSoup(response.content, 'html.parser')    # parses the content (saves the content of the wikipedia site under the city_soup variable)

# retrieve population number for each city and add it to the column population
  city_population = int(city_soup.find("table", class_="vcard").find(string="Population").find_next("td").get_text().replace(",", ""))  # finds pop number, cleans it, turns it into an integer
  populations.append(city_population)  

  # note down the year the data was retrieved
  current_year = (datetime.now()).year
  year_data_retrieved.append(current_year)

cities_population = pd.DataFrame({         # this will be the cities_population dataframe
    "city_name": cities_list,
    "population": populations,
    "year_data_retrieved": year_data_retrieved,
    })  

display(cities_population) 

,city_name,population,year_data_retrieved
0,Berlin,3576873,2024
1,Hamburg,1945532,2024
2,Munich,1512491,2024
3,Cologne,1073096,2024
4,Frankfurt,773068,2024


---
## 2.&nbsp; Sending the information from this notebook to sql 📠
To establish a connection with the SQL database, we need to provide the notebook with the necessary information, which we do using the connection string below. You will need to modify only the password variable, which should match the password you set during MySQL Workbench installation.

In [2]:
schema = "gans"       # name of the database
host = "xxx.x.x.x"    # INSERT YOUR LOCAL HOST HERE (later, it can be changed to the cloud's host)
user = "xxxx"         # name of the user 
password = "xxxx"     # explicitly give SQL password or import from another notebook ("from xxxfile import my_password")
port = xxxx           # insert port number

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'     # this is what connects the python notebook to the sql workbench

In [8]:
cities_info_non_rel.to_sql('cities_info',           # THIS WAY WE PUSH FROM PYTHON TO SQL
                  if_exists='append',       # we don't want to overwrite, only add data to whatever is existent
                  con=connection_string,    # con is the argument we need to connect to sql workbench
                  index=False)    

5

---
## 3. &nbsp; Retrieving information from sql to this notebook & use it for 2nd table 📥

In [9]:
cities_info = pd.read_sql("cities_info", con=connection_string)
cities_info

,city_id,city_name,german_state,latitude,longitude
0,1,Berlin,Berlin,52°31′12″N,13°24′18″E
1,2,Hamburg,Hamburg,53°33′N,10°00′E
2,3,Munich,Bavaria,48°08′15″N,11°34′30″E
3,4,Cologne,North Rhine-Westphalia,50°56′11″N,6°57′10″E
4,5,Frankfurt,Hesse,50°06′38″N,08°40′56″E


In [10]:
# we add the city_id column to our other population dataframe
cities_population["city_id"] = cities_info["city_id"]
cities_population

,city_name,population,year_data_retrieved,city_id
0,Berlin,3576873,2024,1
1,Hamburg,1945532,2024,2
2,Munich,1512491,2024,3
3,Cologne,1073096,2024,4
4,Frankfurt,773068,2024,5


In [11]:
# we drop the city_name column because we don't need it anymore + we reorder the columns to make it more intuititive
cities_population = cities_population[["city_id", "population", "year_data_retrieved"]]
cities_population

,city_id,population,year_data_retrieved
0,1,3576873,2024
1,2,1945532,2024
2,3,1512491,2024
3,4,1073096,2024
4,5,773068,2024


In [12]:
cities_population.dtypes     # later on: we may need to get population to be float!

city_id                int64
population             int64
year_data_retrieved    int64
dtype: object

In [13]:
# we push the second table content to SQL
cities_population.to_sql('cities_population',           # THIS WAY WE PUSH FROM PYTHON TO SQL
                  if_exists='append',       # we don't want to overwrite, only add data to whatever is existent
                  con=connection_string,    # con is the argument we need to connect to sql workbench
                  index=False)    

5

---
## 4.1.&nbsp; Get data from API weather

In [36]:
# from pytz import timezone

def get_weather_data(cities):
  # As we will be working in the cloud, computers can be located anywhere - Let's correct the computer timezone to our local timezone
  berlin_timezone = timezone("Europe/Berlin")
  API_key = "7e5623c79f102b6c08b15c8hjib4cc9l"    # this is not a real one
  weather_items = []

  for city in cities:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    response = requests.get(url)
    json = response.json()

    # Added the time retrieved so we know when the forecast was made
    retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

    # As we are now using the data from our relational database, the city should reflect the city_id and not the city name
    city_id = cities_info.loc[cities_info["city_name"] == city, "city_id"].values[0]  # here we need to retrieve the values, otherwise it shows us the series

    for item in json["list"]:
        weather_item = {
            # Added the city name, so the information is clear when looking at multiple cities
            "city_id": city_id,
            "forecast_time": item.get("dt_txt", None),
            "temperature": item["main"].get("temp", None),
            "feels_like": item["main"].get("feels_like", None),
            "forecast": item["weather"][0].get("main", None),
            "humidity": item["main"].get("humidity", None),
            "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
            "risk_of_rain": item["pop"],
            "snow_in_last_3h": item.get("snow", {}).get("3h", 0),      
            "wind_speed": item["wind"].get("speed", None),
            "data_retrieved_at": retrieval_time
        }

        weather_items.append(weather_item)
  
  weather_df = pd.DataFrame(weather_items)
  weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
  weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])
  weather_df["snow_in_last_3h"] = pd.to_numeric(weather_df["snow_in_last_3h"], downcast="float")

  return weather_df

weather_df = get_weather_data(["Berlin", "Hamburg", "Munich", "Cologne", "Frankfurt"])
weather_df     # we create the new dataframe using the function

,city_id,forecast_time,temperature,feels_like,forecast,humidity,rain_in_last_3h,risk_of_rain,snow_in_last_3h,wind_speed,data_retrieved_at
0,1,2024-03-14 12:00:00,14.42,13.90,Clouds,76,0.00,0.0,0.0,3.46,2024-03-14 12:39:50
1,1,2024-03-14 15:00:00,14.49,13.82,Clouds,70,0.00,0.0,0.0,1.97,2024-03-14 12:39:50
2,1,2024-03-14 18:00:00,13.14,12.44,Clouds,74,0.00,0.0,0.0,2.21,2024-03-14 12:39:50
3,1,2024-03-14 21:00:00,11.34,10.62,Clouds,80,0.00,0.0,0.0,2.29,2024-03-14 12:39:50
4,1,2024-03-15 00:00:00,10.13,9.39,Clouds,84,0.00,0.0,0.0,2.69,2024-03-14 12:39:50
...,...,...,...,...,...,...,...,...,...,...,...
195,5,2024-03-18 21:00:00,11.63,11.07,Rain,85,1.95,1.0,0.0,5.06,2024-03-14 12:39:51
196,5,2024-03-19 00:00:00,10.81,10.14,Clouds,84,0.00,0.8,0.0,5.06,2024-03-14 12:39:51
197,5,2024-03-19 03:00:00,10.22,9.33,Clouds,78,0.00,0.0,0.0,4.87,2024-03-14 12:39:51
198,5,2024-03-19 06:00:00,8.50,6.78,Clouds,86,0.00,0.0,0.0,2.89,2024-03-14 12:39:51


---
## 4.2.&nbsp; Integrate data from API weather to SQL

In [38]:
weather_df.dtypes       # based on these data types, we create the SQL table

city_id                       int64
forecast_time        datetime64[ns]
temperature                 float64
feels_like                  float64
forecast                     object
humidity                      int64
rain_in_last_3h             float64
risk_of_rain                float64
snow_in_last_3h             float32
wind_speed                  float64
data_retrieved_at    datetime64[ns]
dtype: object

In [39]:
# we push the third table content to SQL
weather_df.to_sql("weather",           # THIS WAY WE PUSH FROM PYTHON TO SQL
                  if_exists='append',       # we don't want to overwrite, only add data to whatever is existent
                  con=connection_string,    # con is the argument we need to connect to sql workbench
                  index=False)    

200

## 4.3. Trying out the Cloud function locally:

In [16]:
def extract_city(connection_string):
    return pd.read_sql("cities_info", con=connection_string)
    

In [31]:
cities_df = extract_city(connection_string)
cities_df

,city_id,city_name,german_state,latitude,longitude
0,1,Berlin,Berlin,52°31′12″N,13°24′18″E
1,2,Hamburg,Hamburg,53°33′N,10°00′E
2,3,Munich,Bavaria,48°08′15″N,11°34′30″E
3,4,Cologne,North Rhine-Westphalia,50°56′11″N,6°57′10″E
4,5,Frankfurt,Hesse,50°06′38″N,08°40′56″E


In [37]:
# HERE STARTS THE WEATHER DATA RETRIEVAL FUNCTION:

def extract_city(connection_string):
    return pd.read_sql("cities_info", con=connection_string)

def get_weather_data(cities_df):
  berlin_timezone = timezone("Europe/Berlin")
  API_key = "7e5623c79f102b6c08b15c8hjib4cc9l"    # this is not a real one
  weather_items = []
    
  for city in cities_df["city_name"]:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    response = requests.get(url)
    json = response.json()

    # Added the time retrieved so we know when the forecast was made
    retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

    # As we are now using the data from our relational database, the city should reflect the city_id and not the city name
    city_id = cities_df.loc[cities_df["city_name"] == city, "city_id"].values[0]  # here we need to retrieve the values, otherwise it shows us the series

    for item in json["list"]:
        weather_item = {
            # Added the city name, so the information is clear when looking at multiple cities
            "city_id": city_id,
            "forecast_time": item.get("dt_txt", None),
            "temperature": item["main"].get("temp", None),
            "feels_like": item["main"].get("feels_like", None),
            "forecast": item["weather"][0].get("main", None),
            "humidity": item["main"].get("humidity", None),
            "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
            "risk_of_rain": item["pop"],
            "snow_in_last_3h": item.get("snow", {}).get("3h", 0),      
            "wind_speed": item["wind"].get("speed", None),
            "data_retrieved_at": retrieval_time
        }

        weather_items.append(weather_item)
  
  weather_df = pd.DataFrame(weather_items)
  weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
  weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])
  weather_df["snow_in_last_3h"] = pd.to_numeric(weather_df["snow_in_last_3h"], downcast="float")

  return weather_df

def insert_into_weather(connection_string):
  cities_df = extract_city(connection_string)
  weather_df = get_weather_data(cities_df) # we create the new dataframe using the function
  weather_df.to_sql("weather", 
            if_exists="append",
            con=connection_string,
            index=False) 

In [38]:
insert_into_weather(connection_string)

In [36]:
get_weather_data(cities_df)

,city_id,forecast_time,temperature,feels_like,forecast,humidity,rain_in_last_3h,risk_of_rain,snow_in_last_3h,wind_speed,data_retrieved_at
0,1,2024-03-18 12:00:00,6.40,4.70,Clouds,48,0.00,0.00,0.0,2.33,2024-03-18 11:09:12
1,1,2024-03-18 15:00:00,7.12,5.15,Clouds,42,0.00,0.00,0.0,2.86,2024-03-18 11:09:12
2,1,2024-03-18 18:00:00,5.52,3.29,Clouds,52,0.00,0.00,0.0,2.79,2024-03-18 11:09:12
3,1,2024-03-18 21:00:00,3.50,0.90,Clouds,63,0.00,0.00,0.0,2.76,2024-03-18 11:09:12
4,1,2024-03-19 00:00:00,1.85,-0.84,Clouds,68,0.00,0.00,0.0,2.51,2024-03-18 11:09:12
...,...,...,...,...,...,...,...,...,...,...,...
195,5,2024-03-22 21:00:00,13.25,12.35,Clouds,66,0.00,0.00,0.0,2.29,2024-03-18 11:09:12
196,5,2024-03-23 00:00:00,11.68,10.89,Clouds,76,0.00,0.00,0.0,1.88,2024-03-18 11:09:12
197,5,2024-03-23 03:00:00,10.55,9.80,Clouds,82,0.00,0.00,0.0,2.30,2024-03-18 11:09:12
198,5,2024-03-23 06:00:00,10.68,10.00,Rain,84,0.36,0.38,0.0,2.58,2024-03-18 11:09:12


---
## 4.4. Final function to upload on Google Cloud Platform (GCP)

These were the dependencies needed for this project in the `.txt` file:

```python
functions-framework==3.*
SQLAlchemy==1.4.37
PyMySQL==1.0.2
pandas==1.5.2
requests==2.31.0

In [ ]:
import functions_framework
import pandas as pd
import sqlalchemy
import requests  
from pytz import timezone
from datetime import datetime 

@functions_framework.http
def insert(request):
  connection_string = connection()
  insert_into_weather(connection_string)
  return 'Data successfully added'

def connection():
  connection_name = "flying-dove-416317:europe-west1:wbs-mysql-db"    # this is not a real one
  db_user = "root"
  db_password = "xxxx"   # fill in with your SQL password
  schema_name = "gans"

  driver_name = 'mysql+pymysql'
  query_string = {"unix_socket": f"/cloudsql/{connection_name}"}

  db = sqlalchemy.create_engine(
      sqlalchemy.engine.url.URL(
          drivername = driver_name,
          username = db_user,
          password = db_password,
          database = schema_name,
          query = query_string,
      )
  )
  return db


# HERE STARTS THE WEATHER DATA RETRIEVAL FUNCTION:

def extract_city(connection_string):
    return pd.read_sql("cities_info", con=connection_string)

def get_weather_data(cities_df):
  berlin_timezone = timezone("Europe/Berlin")
  API_key = "7e5623c79f102b6c08b15c8hjib4cc9l"    # this is not a real one
  weather_items = []
    
  for city in cities_df["city_name"]:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    response = requests.get(url)
    json = response.json()

    # Added the time retrieved so we know when the forecast was made
    retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

    # As we are now using the data from our relational database, the city should reflect the city_id and not the city name
    city_id = cities_df.loc[cities_df["city_name"] == city, "city_id"].values[0]  # here we need to retrieve the values, otherwise it shows us the series

    for item in json["list"]:
        weather_item = {
            # Added the city name, so the information is clear when looking at multiple cities
            "city_id": city_id,
            "forecast_time": item.get("dt_txt", None),
            "temperature": item["main"].get("temp", None),
            "feels_like": item["main"].get("feels_like", None),
            "forecast": item["weather"][0].get("main", None),
            "humidity": item["main"].get("humidity", None),
            "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
            "risk_of_rain": item["pop"],
            "snow_in_last_3h": item.get("snow", {}).get("3h", 0),      
            "wind_speed": item["wind"].get("speed", None),
            "data_retrieved_at": retrieval_time
        }

        weather_items.append(weather_item)
  
  weather_df = pd.DataFrame(weather_items)
  weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
  weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])
  weather_df["snow_in_last_3h"] = pd.to_numeric(weather_df["snow_in_last_3h"], downcast="float")

  return weather_df

def insert_into_weather(connection_string):
  cities_df = extract_city(connection_string)
  weather_df = get_weather_data(cities_df)    # we create the new dataframe using the function
  weather_df.to_sql("weather", 
            if_exists="append",
            con=connection_string,
            index=False) 